In [1]:
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
import pandas as pd
from pathlib import Path

In [2]:
client = genai.Client(
    vertexai=True,
    location='europe-west4'
)

In [3]:
pdf_path = Path("../data/ad_annual-report_2024_interactive.pdf")

report = types.Part.from_bytes(
    data=pdf_path.read_bytes(),
    mime_type="application/pdf",
)

In [20]:
prompt: str = """
Answer the following questions:

0. What accounting standards or principles were applied in the preparation of these statements?
1. Do these financial results include any extraordinary or one-time items?
2. Could you specify the time frame these financial statements represent?
3. Are there any transactions with related parties that are documented in the financial statements?
4. Could you detail the principal elements of operating expenses and their classification?
5. Have there been any alterations to accounting policies or practices since prior reporting periods?
6. What process is used to determine and reveal major accounting estimates?
7. What accounting and presentation methods are used for taxes in these financial statements?
8. Are there any contractual commitments or obligations outlined in these statements?
9. Could you describe the nature and extent of the audit or review procedures carried out on these statements?

... using the attached annual statement PDF.
"""

In [ ]:
class AnswerSource(BaseModel):
    citation: str = Field(description="A text citation of the answer in the source document.")
    page: int
    

class AuditAnswer(BaseModel):
    question: str
    answer: str | None
    sources: list[AnswerSource]

class AuditAnswers(BaseModel):
    answers: list[AuditAnswer]



response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=[
        prompt,
        report
    ],
    config=types.GenerateContentConfig(
        temperature=0.2,
        response_mime_type='application/json',
        response_schema=AuditAnswers,
    ),
)
audit_answers: AuditAnswers = response.parsed
audit_answers

AuditAnswers(answers=[AuditAnswer(question='What accounting standards or principles were applied in the preparation of these statements?', answer='These financial statements have been prepared in accordance with International Financial Reporting Standards (IFRSs) as adopted by the EU and also comply with the financial reporting requirements included in Part 9 of Book 2 of the Dutch Civil Code.', sources=[AnswerSource(citation='These financial statements have been prepared in accordance with International Financial Reporting Standards (IFRSs) as adopted by the EU and also comply with the financial reporting requirements included in Part 9 of Book 2 of the Dutch Civil Code.', page=264)]), AuditAnswer(question='Do these financial results include any extraordinary or one-time items?', answer='The financial results do include unusual items, related to impairment, losses and restructuring charges.', sources=[AnswerSource(citation='This decrease resulted from unusual items, related to impairm

In [18]:
df = pd.DataFrame(
    [
        answer.model_dump() for answer in audit_answers.answers
    ]
)
df

,question,answer,sources
0,What accounting standards or principles were a...,These financial statements have been prepared ...,[{'citation': 'These financial statements have...
1,Do these financial results include any extraor...,The financial results do include unusual items...,[{'citation': 'This decrease resulted from unu...
2,Could you specify the time frame these financi...,The financial statements cover the 52-week per...,[{'citation': 'The financial year 2024 consist...
3,Are there any transactions with related partie...,"Yes, Note 31 to the financial statements conta...",[{'citation': 'For more details on the remuner...
4,Could you detail the principal elements of ope...,The principal elements of operating expenses a...,"[{'citation': 'The aggregate of cost of sales,..."
5,Have there been any alterations to accounting ...,"Yes, there have been some changes in accountin...",[{'citation': 'Comparative amounts have been r...
6,What process is used to determine and reveal m...,The document indicates that preparing financia...,[{'citation': 'The preparation of financial st...
7,What accounting and presentation methods are u...,The financial statements use the sum of curren...,[{'citation': 'Income tax expense represents t...
8,Are there any contractual commitments or oblig...,"Yes, there are contractual commitments and obl...","[{'citation': 'As of December 29, 2024, Ahold ..."
9,Could you describe the nature and extent of th...,"The audit was performed with a high, but not a...",[{'citation': 'Our audit has been performed wi...
